In [1]:
import torch
import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.enabled = False 
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

PATH="/export/data4/vzhekova/biases-data/Test_Fr"

# Translation English-French

Sentence: 
- My friend is a student.

In [2]:
%cd $PATH

/export/data4/vzhekova/biases-data/Test_Fr


In [3]:
# Dividing tokenized text into subword units

!subword-nmt apply-bpe -c bpecodes.en-fr < test.en-fr.en > sw.test.en-fr.en
!subword-nmt apply-bpe -c bpecodes.en-fr < test.en-fr.fr > sw.test.en-fr.fr

print('Finished subword.')

Finished subword.


In [4]:
!fairseq-preprocess \
    --source-lang en \
    --target-lang fr \
    --testpref sw.test.en-fr \
    --srcdict /export/data4/vzhekova/biases-data/En-Fr/wmt14.en-fr.joined-dict.transformer/dict.en.txt \
    --tgtdict /export/data4/vzhekova/biases-data/En-Fr/wmt14.en-fr.joined-dict.transformer/dict.fr.txt \
    --destdir data-bin_en-fr \
    --workers 8

2023-03-22 15:46:06 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data-bin_en-fr', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=False, scoring='bleu', seed=1, source_lang='en', srcdict='/ex

In [5]:
%cd $PATH

/export/data4/vzhekova/biases-data/Test_Fr


- Beam search

In [6]:
NBEST = 10
BEAM = 10

# Generate N hypothesis
!fairseq-generate data-bin_en-fr  \
    --task translation \
    --source-lang en \
    --target-lang fr \
    --path /export/data4/vzhekova/biases-data/En-Fr_MuST-C/checkpoints/checkpoint_best.pt \
    --beam $BEAM \
    --nbest $NBEST \
    --batch-size 256 \
    --remove-bpe=subword_nmt > en-fr.decode_Beam_10.log

2023-03-22 15:46:24 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name'

# Backtranslation French-English

- Extract hypothesis

In [7]:
!grep ^H en-fr.decode_Beam_10.log | sed 's/^H-//g' | cut -f 3 | sed 's/ @@//g' > ./hyp.txt

In [8]:
!head ./hyp.txt

Mon ami est étudiant .
Mon ami est un étudiant .
Mon ami est étudiante .
Mon ami est un élève .
Mon ami est une étudiante .
Mon ami est élève .
Mon ami est étudiants .
Mon ami est un étudiante .
Mon ami est un étudiant .
Mes amis sont étudiants .


In [16]:
# Dividing tokenized text into subword units

!subword-nmt apply-bpe -c bpecodes.fr-en < hyp.fr > sw.hyp.fr

print('Finished subword.')

Finished subword.


In [17]:
!head sw.hyp.fr

Mon ami est un étudiant .
Mon ami est étudiant .
Mon am@@ ie est étudi@@ ante .
Mon ami est une étudi@@ ante .
Ma cop@@ ine est étudi@@ ante .
Mon ami est un élève .
Mon ami étudie .
Mon ami est étudi@@ ante .
Mon am@@ ie étudie .
Mon ami est un étudiant !


In [18]:
# !!! Copy the dict.fr to data-bin for translation
!fairseq-preprocess \
    --source-lang fr \
    --target-lang en \
    --only-source \
    --testpref sw.hyp \
    --srcdict /export/data4/vzhekova/biases-data/Fr-En/training-data/wmt14_fr_en/data-bin/dict.fr.txt \
    --destdir data-bin_fr-en \
    --workers 8

2023-03-22 11:12:47 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data-bin_fr-en', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=True, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_config_path=None, reset_logging=False, scoring='bleu', seed=1, source_lang='fr', srcdict='/exp

- Translate hypothesis back to En

In [19]:
NBEST = 10
BEAM = 10

# Generate N hypothesis
!fairseq-generate data-bin_fr-en  \
    --task translation \
    --source-lang fr \
    --target-lang en \
    --path /export/data4/vzhekova/biases-data/Fr-En_MuST-C/checkpoints/checkpoint_best.pt \
    --beam $BEAM \
    --nbest $NBEST \
    --batch-size 256 \
    --remove-bpe=subword_nmt > fr-en.decode_Beam_10_backtranslation.log

2023-03-22 11:13:00 | INFO | fairseq_cli.generate | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name'